In [1]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [8]:
function kalman_filter_graph()
    model = Model(; message_gate = DefaultMessageGate() + LoggerMessageGate())
    
    x_prev = datavar(model, :x_prior, Normal{Float64})
    add_1  = constvar(model, :add_1, 1.0)
    x      = randomvar(model, :x)
    
    x_prev_add = make_node(model, +, x_prev, add_1, x)
    
    noise = constvar(model, :noise, Normal(0.0, sqrt(200.0)))
    y     = datavar(model, :y, Dirac{Float64})
    
    add_x_and_noise = make_node(model, +, x, noise, y)
    
    activate!(model)
    
    return x_prev, x, y
end

function kalman_logger(data)
    N = length(data)
    
    x_prev, x, y = kalman_filter_graph()

    link(x_prev, getmarginal(x))
    
    update!(x_prev, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman_logger (generic function with 1 method)

In [9]:
N = 10
data = collect(1:N) + sqrt(200.0) * randn(N);

In [10]:
kalman_logger(data);

From variable in2 of node + => Normal{Float64}(μ=-1.0, σ=100.0)
From variable out of node + => Normal{Float64}(μ=11.198471888097066, σ=14.142135623730951)
From variable in2 of node + => Normal{Float64}(μ=9.959286164801044, σ=14.002800840280099)
